In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
torch.cuda.get_device_name(0)

'NVIDIA A100-SXM4-40GB'

In [ ]:
import json
import random
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
!pip install qwen_vl_utils
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 53.7 MB/s eta 0:00:00


In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install -e .[torch,bitsandbytes]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 345, done.
remote: Counting objects: 100% (345/345), done.
remote: Compressing objects: 100% (280/280), done.
remote: Total 345 (delta 85), reused 159 (delta 49), pack-reused 0 (from 0)
Receiving objects: 100% (345/345), 9.60 MiB | 30.61 MiB/s, done.
Resolving deltas: 100% (85/85), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install deepspeed
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 30.0 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.16.4-py3-none-any.whl size=1562649 sha256=a30fbe206685c5765d95385318384c48fd7bff996bf03280922418ce42a4ab85
  Stored in directory: /root/.cache/pip/wheels/b3/6c/e5/ccad75c8ade9cb21e74721affd6d17820b1806249aac34f7f0
Successfully built deepspeed
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 109.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187831595 sha256=58853b28a5a926cae14402bfd8d4d93a45ebf8f9e79533f37ab09d0d77a99c05
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built

In [ ]:
!pip install rouge_chinese

### build training dataset

In [ ]:
with open('/content/drive/MyDrive/CrowdLLM/new_datasets/vlm_r1_train.json', 'r') as f:
  data = json.load(f)

tuning_data = []

for entry in data:
  img_path = entry["image"]
  obj_type = entry["type"]
  obj_count = entry["solution"]
  problem = entry["problem"]
  conversation = {
    "conversations": [
      {
        "role": "user",
        "content": f"<image>\n{problem}. Please provide your answer as a number in the following format: count"
      },
      {
        "role": "assistant",
        "content": str(obj_count)
      }
    ],
    "image_paths": ['/content/drive/MyDrive/CrowdLLM/new_datasets/vlm_r1_train/'+img_path]
  }
  tuning_data.append(conversation)

random.shuffle(tuning_data)

print(tuning_data[0])

drive_path = "/content/drive/MyDrive/CrowdLLM/crowd_train.jsonl"
llama_path = "/content/LLaMA-Factory/data/crowd_train.jsonl"
with open(llama_path, "w", encoding="utf-8") as f:
  for item in tuning_data:
    f.write(json.dumps(item, ensure_ascii=False) + "\n")

{'conversations': [{'role': 'user', 'content': '<image>\nPlease count the number of people in the image. Please provide your answer as a number in the following format: count'}, {'role': 'assistant', 'content': '78'}], 'image_paths': ['/content/drive/MyDrive/CrowdLLM/new_datasets/vlm_r1_train/GTA_Head/train/scene_22/img1/000045.jpg']}


#build test dataset

In [ ]:
with open('/content/drive/MyDrive/CrowdLLM/new_datasets/vlm_r1_test.json', 'r') as f:
    test_data = json.load(f)

eval_data = []

for entry in test_data:
    img_path = entry["image"]
    obj_type = entry["type"]
    obj_count = entry["solution"]  # This is your ground-truth count
    problem = entry["problem"]

    eval_entry = {
      "conversations": [
        {
          "role": "user",
          "content": f"<image>\n{problem}. Please provide your answer as a number in the following format: count"
        },
        {
          "role": "assistant",
          "content": str(obj_count)  # Ground truth answer
        }
      ],
      "image_paths": [
          "/content/drive/MyDrive/CrowdLLM/new_datasets/vlm_r1_test/" + img_path
      ]
    }
    eval_data.append(eval_entry)

print(eval_data[0])

drive_path = "/content/drive/MyDrive/CrowdLLM/crowd_eval.jsonl"
llama_path = "/content/LLaMA-Factory/data/crowd_eval.jsonl"
with open(llama_path, "w", encoding="utf-8") as f:
    for item in eval_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

{'conversations': [{'role': 'user', 'content': '<image>\nPlease count the number of sheep in the image. Please provide your answer as a number in the following format: count'}, {'role': 'assistant', 'content': '71'}], 'image_paths': ['/content/drive/MyDrive/CrowdLLM/new_datasets/vlm_r1_test/Aerial_Sheep.v1-v1.coco/test/DJI_0039_MOV-64_jpg.rf.4931d1329fc1401f10da1ba1f6021934.jpg']}


# Add to LLaMA-Factory/data/dataset_info.json


In [ ]:
  # "crowd_train": {
  #   "file_name": "crowd_train.jsonl",
  #   "formatting": "sharegpt",
  #   "columns": {
  #     "messages": "conversations",
  #     "images": "image_paths"
  #   },
  #   "tags": {
  #     "role_tag": "role",
  #     "content_tag": "content",
  #     "user_tag": "user",
  #     "assistant_tag": "assistant"
  #   }
  # },
  # "crowd_eval": {
  #   "file_name": "crowd_eval.jsonl",
  #   "formatting": "sharegpt",
  #   "columns": {
  #     "messages": "conversations",
  #     "images": "image_paths"
  #   },
  #   "tags": {
  #     "role_tag": "role",
  #     "content_tag": "content",
  #     "user_tag": "user",
  #     "assistant_tag": "assistant"
  #   }
  # },

# Training

In [ ]:
# Either launch llama factory webui
%cd /content/LLaMA-Factory/
!GRADIO_SHARE=1 llamafactory-cli webui

/content/LLaMA-Factory
2025-03-27 22:19:24.528837: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-27 22:19:24.547620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743113964.569912   23826 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743113964.576639   23826 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-27 22:19:24.599386: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to 

In [ ]:
# Or use command to do training/evaluation
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path Qwen/Qwen2.5-VL-3B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen2_vl \
    --flash_attn auto \
    --dataset_dir data \
    --dataset crowd_train \
    --cutoff_len 2048 \
    --learning_rate 5e-05 \
    --num_train_epochs 1.0 \
    --max_samples 100000 \
    --per_device_train_batch_size 2 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 100 \
    --warmup_steps 0 \
    --packing False \
    --report_to none \
    --output_dir saves/Qwen2.5-VL-3B-Instruct/lora/train_2025-03-27-22-19-42 \
    --bf16 True \
    --plot_loss True \
    --trust_remote_code True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --optim adamw_torch \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --freeze_vision_tower False

2025-03-27 22:20:27.418024: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-27 22:20:27.436577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743114027.458381   24156 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743114027.465010   24156 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-27 22:20:27.487540: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Evaluation

In [ ]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path Qwen/Qwen2.5-VL-3B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --quantization_method bitsandbytes \
    --template qwen2_vl \
    --flash_attn auto \
    --dataset_dir data \
    --eval_dataset crowd_eval \
    --cutoff_len 1024 \
    --max_samples 100000 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves/Qwen2.5-VL-3B-Instruct/lora/qwen2.5-vl-3b-instruct-lora-visiontrainable-eval \
    --trust_remote_code True \
    --do_predict True \
    --adapter_name_or_path saves/Qwen2.5-VL-3B-Instruct/lora/train_2025-03-27-22-19-42

2025-03-28 00:25:18.680114: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-28 00:25:18.698397: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743121518.720289   55838 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743121518.726952   55838 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-28 00:25:18.749302: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Store results

In [ ]:
import shutil

In [ ]:
# Save models to Google
shutil.copytree('/content/LLaMA-Factory/saves/Qwen2.5-VL-3B-Instruct/lora/train_2025-03-27-21-03-34',
                '/content/drive/MyDrive/CrowdLLM/qwen2.5-vl-3b-instruct-lora')

'/content/drive/MyDrive/CrowdLLM/qwen2.5-vl-3b-instruct-lora'

In [ ]:
# Save predictions
shutil.copytree('/content/LLaMA-Factory/saves/Qwen2.5-VL-3B-Instruct/lora/qwen2.5-vl-3b-instruct-lora-eval',
                '/content/drive/MyDrive/CrowdLLM/qwen2.5-vl-3b-instruct-lora-eval')

'/content/drive/MyDrive/CrowdLLM/qwen2.5-vl-3b-instruct-lora-eval'

In [ ]:
shutil.copytree('/content/LLaMA-Factory/saves/Qwen2.5-VL-3B-Instruct/lora/train_2025-03-27-22-19-42',
                '/content/drive/MyDrive/CrowdLLM/qwen2.5-vl-3b-instruct-lora-visiontrainable')

'/content/drive/MyDrive/CrowdLLM/qwen2.5-vl-3b-instruct-lora-visiontrainable'

In [ ]:
shutil.copytree('/content/LLaMA-Factory/saves/Qwen2.5-VL-3B-Instruct/lora/qwen2.5-vl-3b-instruct-lora-visiontrainable-eval',
                '/content/drive/MyDrive/CrowdLLM/qwen2.5-vl-3b-instruct-lora-visiontrainable-eval')

'/content/drive/MyDrive/CrowdLLM/qwen2.5-vl-3b-instruct-lora-visiontrainable-eval'